In [ ]:
#NEW processing for GT test set 
import nltk
import json
import os
from collections import defaultdict


formulamapfile = "formulaimgmap.json"
with open(formulamapfile, 'r') as file:
    formulaimgmap = json.load(file)


# extripclipfile = "allTripClipScore/extractedTripXlan.json"
# with open(extripclipfile, 'r') as file:
    # extripclipscores = json.load(file)

# modelname = 'xlan'
# dirname = "output_test_"+modelname+"/"
# files = os.listdir(dirname)
modelname = 'GT'
C1 = 0
C2=0
C3=0
grcounts = []
doneprocessing = {}
groundingmap = defaultdict(list)
captionmap = {}
img2triples = defaultdict(list)
testrel2formclips = defaultdict(list)

with open('allTripletsGT_Test.json', 'r') as file:
    img2triplesGT = json.load(file)
    
with open('captionGenTest/test_gt_caption.json') as file:
    allcaptionGT = json.load(file)
# for filename in files:
#     if filename.find(".ipynb_checkpoints")>=0:
#         continue
#     separator = "Caption"
#     if not os.path.isfile(dirname+filename):
#         continue
#     C3=C3+1
#     #if C3>1:
#     #    break
#     ifile = open(dirname+filename)
#     alltriplets = []
#     allcaptions = []
#     imageids = []
#     testrel2formclips = defaultdict(list)
#     for ln in ifile:
#         if ln.find(separator)==0:
#             cparts = ln.strip().split(":")
#             incap = {}
#             ln1 = ifile.readline()
#             p1 = ln1.find("Imageid: ")
#             p2 = ln1.find(",")
#             parts = ln1[p1:p2].split(":")
#             if len(parts) < 2:
#                 continue
#             imageid = parts[1].strip()
#             if str(imageid) in doneprocessing:
#                 continue
#             p1 = ln1.find("extracted triplets: [")
#             p2 = ln1.find("]")
#             parts = ln1[p1:p2].split("[")
#             if len(parts)<2:
#                 continue
#             extr = parts[1].split(",")
#need to start from here
for imageid in img2triplesGT:
    extr = img2triplesGT[imageid]
    cparts = allcaptionGT[imageid] #taking first caption
    # for i in range(0,len(extr),1):
        # extr[i] = extr[i].strip()
        # extr[i] = extr[i][1:len(extr[i])-1]

    if imageid in img2triples:
        continue
    for e1 in extr:
        img2triples[imageid].append(e1)
    pairs = []
    pairclipscores = {}
    newobjs = []
    if len(extr)==1:
        #C1 = C1+1
        tokens = nltk.word_tokenize(cparts[0])
        tagged = nltk.pos_tag(tokens)
        sparts = extr[0].split()
        for tg in tagged:
            if tg[1]=='NN':
                if tg[0]!=sparts[0] and tg[0]!=sparts[len(sparts)-1]:
                    newobjs.append(tg[0])
        #if len(newobjs)==0:
        #    C1=C1+1
        P1 = extr[0].split()
        pairs.append(P1[0])
        pairs.append(P1[len(P1)-1])
        #pairclipscores[P1[0]+":"+P1[len(P1)-1]] = clipscorestripls[extr[0]]
    elif len(extr)==2:
        #C2=C2+1
        P1 = extr[0].split()
        P2 = extr[1].split()
        pairs.append(P1[0])
        pairs.append(P1[len(P1)-1])
        pairs.append(P2[0])
        pairs.append(P2[len(P2)-1])
        #pairclipscores[P1[0]+":"+P1[len(P1)-1]] = clipscorestripls[extr[0]]
        #pairclipscores[P2[0]+":"+P2[len(P2)-1]] = clipscorestripls[extr[1]]
    else:
        #ofile.write(str(imageid)+":"+cparts[1]+":"+",".join(extr)+"\n")
        modtriples = []
        for e in extr:
            parts=e.split()
            captionwords = cparts[0].split()
            w1=False
            w2=False
            for c in captionwords:
                if parts[0]==c:
                    w1=True
                if parts[len(parts)-1]==c:
                    w2=True
            if w1 and w2:
                modtriples.append(e)

        if len(modtriples)==0:
            #C1=C1+1
            tokens = nltk.word_tokenize(cparts[0])
            tagged = nltk.pos_tag(tokens)
            newobjs = []
            for tg in tagged:
                if tg[1]=='NN':
                    newobjs.append(tg[0])
        if len(modtriples)>=2:
            for m in modtriples:
                P1 = m.split()
                pairs.append(P1[0])
                pairs.append(P1[len(P1)-1])

    cnt=0
    newobjs = list(set(newobjs))
    listofforms = []
    for fr in formulaimgmap:
        parts = fr.split(":")
        gr = 0
        P1 = parts[0].split()
        P2 = parts[1].split()

        tmp = []
        tmp.append(P1[0])
        tmp.append(P1[len(P1)-1])
        tmp.append(P2[0])
        tmp.append(P2[len(P2)-1])
        if len(set(tmp))<3:
            continue

        m1 = False
        m2 = False

        for i in range(0,len(pairs)-1,2):
            if P1[0]==pairs[i] and P1[len(P1)-1]==pairs[i+1]:
                m1 = True
                break
        for i in range(0,len(pairs)-1,2):
            if P2[0]==pairs[i] and P2[len(P2)-1]==pairs[i+1]:
                m2 = True
                break
        if m1 and m2:
            #testrel2formclips[fr].append(pairclipscores[P1[0]+":"+P1[len(P1)-1]])
            #testrel2formclips[fr].append(pairclipscores[P2[0]+":"+P2[len(P2)-1]])
            cnt=cnt+1
            listofforms.append(fr)
        else:
            if m1 and ((P2[0] in newobjs) and (P2[len(P2)-1] in newobjs)):
                cnt=cnt+1
                #testrel2formclips[fr].append(pairclipscores[P1[0]+":"+P1[len(P1)-1]])
                #testrel2formclips[fr].append(0)
                listofforms.append(fr)
            if m2 and ((P1[0] in newobjs) and (P1[len(P1)-1] in newobjs)):
                cnt=cnt+1
                #testrel2formclips[fr].append(0)
                #testrel2formclips[fr].append(pairclipscores[P2[0]+":"+P2[len(P2)-1]])
                listofforms.append(fr)

    groundingmap[str(imageid)]=listofforms
    #grcounts.append(cnt)
    doneprocessing[str(imageid)]=1     
    captionmap[str(imageid)]=cparts[0]
    # ifile.close()
with open('groundingmap-{}.json'.format(modelname), 'w') as fp:
    json.dump(groundingmap, fp) 
with open('captionmap-{}.json'.format(modelname), 'w') as fp:
    json.dump(captionmap, fp) 
# with open('testrel2formsclip-{}.json'.format(modelname), 'w') as fp:
   # json.dump(testrel2formclips, fp) 
with open('img2trip-{}.json'.format(modelname), 'w') as fp:
   json.dump(img2triples, fp) 